In [1]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["OPENAI_API_KEY"] = "sk-c1h5RWNyx6WxgiNEF2D01f2cF1A042De95F94b6b3b0cB5C3"

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", base_url="https://api3.wlai.vip/v1/")

In [2]:
import base64

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [3]:
from typing import List, Optional

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator


class Result(BaseModel):
    """
    The result of the image locator.
    """
    index: int= Field(
        default=None, 
        description="the index of the upper-left corner of the bbox")
    
    reason: str = Field(
        default=None,
        description="the reason why the index is chosen"
    )

# Set up a parser
parser = PydanticOutputParser(pydantic_object=Result)

In [4]:
from langchain_core.messages import HumanMessage

system_instruction = """You are a helpful image locator who can understand the text descriptions I give you about a specific object. You need to answer which bounding box (bbox) in the image contains the target object from the text description. You only need to provide the index of the upper-left corner of the bbox and explain why you chose it. "Wrap the output in `json` tags\n{format_instructions}"
"""
description = "if you are facing the three windows on one wall it is the window on the left."

content = [{"type": "text", "text": description}]
for i in range(2):
    content.append({"type": "image_url", "image_url": {
        "url": f"data:image/png;base64,{encode_image_to_base64(f'view_{i}.png')}"
    }})

message = [
    ("system", system_instruction),
    HumanMessage(content=content)
]

prompt = ChatPromptTemplate.from_messages(message).partial(format_instructions=parser.get_format_instructions())
# prompt.invoke({})

FileNotFoundError: [Errno 2] No such file or directory: 'view_0.png'

In [7]:
chain = prompt | model | parser
result = chain.invoke({})
print(type(result))
print("result", result)

NameError: name 'prompt' is not defined

In [6]:
model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-0350e9db-9db8-4776-9da8-2850e7e9121f-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})